In [10]:
#2/7/24 Random Forest Model Creation for Chemopy Descriptors of NR-AR Dataset

import pandas as pd
import numpy as np
import math
import os
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#Data import
seed = 82

datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/structdata/endocrine_redux/Chemopy/NR-AR/'
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df.drop(columns=['SMILES'], inplace=True)
        
        #drops 90% of negative columns to resolve class imbalance
        ytrain = df.iloc[:, 0].values
        reps = 0
        todrop = []
        for item in ytrain:
            if reps % 10 != 0 and item == 0:
                todrop.append(reps)
            reps = reps + 1

        df = df.drop(todrop)
        df = df.dropna(axis=1)
        datasets.append(df)
        index.append(file)

In [11]:
loops = 0
max = 0

for df in datasets:
    print(df.isnull().values.any())
    print(index[loops])
    datasets[loops] = datasets[loops].replace([np.inf, -np.inf], np.nan).dropna()
    xtrain = df.iloc[:, 1:].values
    nrow = 0
    ncol = 0
    for item in xtrain:
        ncol = 0
        for thing in item:
            if math.isinf(thing) == True:
                print(ncol, nrow, '\n')
            ncol = ncol + 1
        nrow = nrow + 1
    loops = loops + 1
    
    ytrain = df.iloc[:, 0].values
    tc = 0
    fc = 0
    for item in ytrain:
        if item == 0:
            fc = fc + 1
        else:
            tc = tc + 1
    
    print(tc, fc)

False
NR-AR_moe.csv
303 684
False
NR-AR_molprop.csv
303 684
False
NR-AR_charge.csv
0 977 

303 684
False
NR-AR_constitution.csv
303 684
False
NR-AR_estate.csv
303 684
False
NR-AR_connectivity.csv
303 684
False
NR-AR_basak.csv
303 684
False
NR-AR_kappa.csv
303 684


In [12]:
#function to calculate various metrics, outputs a list of various metrics with a consistent index
def metriccalc(preds, ytrain):
    correctcount = 0
    fpcount = 0
    tpcount = 0
    tncount = 0
    fncount = 0
    testpos = 0
    testneg = 0
    
    #loop through each item in the predictions, logging positives, negatives and tn/tp/fn/tp
    iterations = 0
    for value in preds:
        testscore = ytrain[iterations]
        if value == 1:
            if testscore != 0:
                testpos = testpos + 1
                correctcount = correctcount + 1
                tpcount = tpcount + 1
            else:
                fpcount = fpcount + 1
                testneg = testneg + 1
        else:
            if testscore != 0:
                testpos = testpos + 1
                fncount = fncount + 1
            else:
                testneg = testneg + 1
                correctcount = correctcount + 1
                tncount = tncount + 1

        iterations = iterations + 1
    
    #calculate a wide swathe of metrics
    netfn = fncount / (fncount + tncount)
    nettn = tncount / (fncount + tncount)
    netacc = correctcount / (fpcount + fncount + tpcount + tncount)
    posacc = tpcount / testpos
    negacc = tncount / testneg
    netfp = fpcount / (fpcount + tpcount)
    nettp = tpcount / (tpcount + fpcount)

    fpr = fpcount / (fpcount + tncount)
    tpr = tpcount / (tpcount + fncount)



    f1 = (2 * tpcount) / ((2 * tpcount) + fpcount + fncount)


    tp = tpcount
    fp = fpcount
    tn = tncount
    fn = fncount

    
    temp = math.sqrt((fp + tn) * (tp + fp) * (tp + fn) * (tn + fn))
    if temp == 0:
        return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    mcc = ((tp * tn) - (fp * fn)) / temp



    temp = (( ( (tp + fp) * (fp + tn) ) + ( (tp + fn) * (fn + tn) ) ))
    if temp == 0:
        return 0
    kapp =  ( 2 * ((tp * tn) - (fn * fp)) ) / temp
    
    metriclist = [testpos, testneg, fn, tn, tp, fp, netacc, posacc, negacc, fpr, tpr, f1, mcc, kapp]
    
    return metriclist

In [13]:
loops = 0
from sklearn import preprocessing
import numpy as np
for targetdata in datasets:
    trainset, testset = train_test_split(targetdata, test_size=0.5, random_state=seed)
    xtrain = trainset.iloc[:, 1:]
    ytrain = trainset.iloc[:, 0].values
    xtest = testset.iloc[:, 1:]
    ytest = testset.iloc[:, 0].values

#    scaler = preprocessing.StandardScaler().fit(xtrain)
#    xtrain = scaler.transform(xtrain)
    
#    scaler = preprocessing.StandardScaler().fit(xtest)
#    xtest = scaler.transform(xtest)
    
    mcclist = []
    maxval = 0
    for mtry in range(1, 50):
        #using mtry as the adjusted hyperparameter creates a series of random forests
        rf = RandomForestClassifier(n_estimators=mtry, criterion='entropy', max_depth=None, 
                                min_samples_split=2, min_samples_leaf=1, 
                                min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                bootstrap=True, oob_score=False, n_jobs= 4, random_state=seed, 
                                verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xtest)
        #calculate metric (mcc)
        mcc = metriccalc(preds, ytest)[12]
        mcclist.append(mcc)
        if mcc > maxval:
            maxval = mcc
            bestmetrics = mtry
    #store best model for the given fold and plot the metric vs mcc value
    rf = RandomForestClassifier(n_estimators=bestmetrics, criterion='entropy', max_depth=None, 
                                min_samples_split=2, min_samples_leaf=1, 
                                min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                bootstrap=True, oob_score=False, n_jobs= 4, random_state=seed, 
                                verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
    print('best mcc of', maxval, 'with an mtry of', bestmetrics)
    #plt.plot(mcclist)

    #plt.xlabel('mtry')
    #plt.ylabel('mcc')
    #plt.legend()
    #plt.show()

    model= rf.fit(xtrain, ytrain)
    preds = model.predict(xtest)
    results = metriccalc(preds, ytest)
    print('for dataset', index[loops], 'validation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1], '\n')
    print('fn count =', results[2])
    print('tn count =', results[3])
    print('tp count =', results[4])
    print('fp count =', results[5], '\n')
    print('net accuracy =', results[6])
    print('positive accuracy =', results[7])
    print('negative accuracy =', results[8], '\n')
    print('fpr =', results[9])
    print('tpr =', results[10], '\n')
    print('f1 score =',results[11])
    print('mcc =',results[12])
    print('cohen Kappa =',results[13], '\n \n')
    loops = loops + 1

best mcc of 0.6282352452375785 with an mtry of 16
for dataset NR-AR_moe.csv validation metrics of:
positives in data 143
negatives in data 351 

fn count = 58
tn count = 337
tp count = 85
fp count = 14 

net accuracy = 0.854251012145749
positive accuracy = 0.5944055944055944
negative accuracy = 0.9601139601139601 

fpr = 0.039886039886039885
tpr = 0.5944055944055944 

f1 score = 0.7024793388429752
mcc = 0.6282352452375785
cohen Kappa = 0.6101453405528641 
 

best mcc of 0.5278819272220475 with an mtry of 18
for dataset NR-AR_molprop.csv validation metrics of:
positives in data 143
negatives in data 351 

fn count = 67
tn count = 328
tp count = 76
fp count = 23 

net accuracy = 0.8178137651821862
positive accuracy = 0.5314685314685315
negative accuracy = 0.9344729344729344 

fpr = 0.06552706552706553
tpr = 0.5314685314685315 

f1 score = 0.628099173553719
mcc = 0.5278819272220475
cohen Kappa = 0.51268167569108 
 

best mcc of 0.24768160996076158 with an mtry of 18
for dataset NR-AR_char

In [61]:
X

NameError: name 'X' is not defined